# This is a demo to generate new wiki articles
## using LSTM and RNN 

In [1]:
import numpy as np 
import random  
import tensorflow as tf 
import datetime 

In [3]:
text = open('./data/wikitext-2-raw/wiki.test.raw').read()
print('text length in number of characters:', len(text))

print('head of text:')
print(text[:1000]) #all tokenized words, stored in a list called text

text length in number of characters: 1288556
head of text:
 
 = Robert Boulter = 
 
 Robert Boulter is an English film , television and theatre actor . He had a guest @-@ starring role on the television series The Bill in 2000 . This was followed by a starring role in the play Herons written by Simon Stephens , which was performed in 2001 at the Royal Court Theatre . He had a guest role in the television series Judge John Deed in 2002 . In 2004 Boulter landed a role as " Craig " in the episode " Teddy 's Story " of the television series The Long Firm ; he starred alongside actors Mark Strong and Derek Jacobi . He was cast in the 2005 theatre productions of the Philip Ridley play Mercury Fur , which was performed at the Drum Theatre in Plymouth and the Menier Chocolate Factory in London . He was directed by John Tiffany and starred alongside Ben Whishaw , Shane Zaza , Harry Kent , Fraser Ayres , Sophie Stanton and Dominic Hall . 
 In 2006 , Boulter starred alongside Whishaw in the play 

In [7]:
chars = sorted(list(set(text)))
char_size = len(chars)
print('number of characters:', char_size)
print(chars)

number of characters: 259
['\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '^', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '£', '¥', '©', '°', '½', 'Á', 'Æ', 'É', '×', 'ß', 'à', 'á', 'ã', 'ä', 'å', 'æ', 'ç', 'è', 'é', 'ê', 'ë', 'í', 'î', 'ñ', 'ó', 'ô', 'ö', 'ú', 'ü', 'ć', 'č', 'ě', 'ī', 'ł', 'Ō', 'ō', 'Š', 'ū', 'ž', 'ǐ', 'ǔ', 'ǜ', 'ə', 'ɛ', 'ɪ', 'ʊ', 'ˈ', 'ː', '̍', '͘', 'Π', 'Ω', 'έ', 'α', 'β', 'δ', 'ε', 'ι', 'λ', 'μ', 'ν', 'ο', 'π', 'ς', 'σ', 'τ', 'υ', 'ω', 'ό', 'П', 'в', 'д', 'и', 'к', 'н', 'א', 'ב', 'י', 'ל', 'ר', 'ש', 'ת', 'ا', 'ت', 'د', 'س', 'ك', 'ل', 'و', 'ڠ', 'ग', 'न', 'र', 'ल', 'ष', 'ु', 'े', 'ो', '्', 'ả', 'ẩ', '‑', '–', '—', '’', '“

In [12]:
char2id = dict((c, i) for i, c in enumerate(chars))
id2char = dict((i, c) for i, c in enumerate(chars))
print(id2char)

{0: '\n', 1: ' ', 2: '!', 3: '"', 4: '#', 5: '$', 6: '%', 7: '&', 8: "'", 9: '(', 10: ')', 11: '*', 12: '+', 13: ',', 14: '-', 15: '.', 16: '/', 17: '0', 18: '1', 19: '2', 20: '3', 21: '4', 22: '5', 23: '6', 24: '7', 25: '8', 26: '9', 27: ':', 28: ';', 29: '<', 30: '=', 31: '>', 32: '?', 33: '@', 34: 'A', 35: 'B', 36: 'C', 37: 'D', 38: 'E', 39: 'F', 40: 'G', 41: 'H', 42: 'I', 43: 'J', 44: 'K', 45: 'L', 46: 'M', 47: 'N', 48: 'O', 49: 'P', 50: 'Q', 51: 'R', 52: 'S', 53: 'T', 54: 'U', 55: 'V', 56: 'W', 57: 'X', 58: 'Y', 59: 'Z', 60: '[', 61: ']', 62: '^', 63: 'a', 64: 'b', 65: 'c', 66: 'd', 67: 'e', 68: 'f', 69: 'g', 70: 'h', 71: 'i', 72: 'j', 73: 'k', 74: 'l', 75: 'm', 76: 'n', 77: 'o', 78: 'p', 79: 'q', 80: 'r', 81: 's', 82: 't', 83: 'u', 84: 'v', 85: 'w', 86: 'x', 87: 'y', 88: 'z', 89: '£', 90: '¥', 91: '©', 92: '°', 93: '½', 94: 'Á', 95: 'Æ', 96: 'É', 97: '×', 98: 'ß', 99: 'à', 100: 'á', 101: 'ã', 102: 'ä', 103: 'å', 104: 'æ', 105: 'ç', 106: 'è', 107: 'é', 108: 'ê', 109: 'ë', 110: 'í'

In [13]:
def sample(prediction):
    #Samples are uniformly distributed over the half-open interval 
    r = random.uniform(0,1)
    #store prediction char
    s = 0
    #since length > indices starting at 0
    char_id = len(prediction) - 1
    #for each char prediction probabilty
    for i in range(len(prediction)):
        #assign it to S
        s += prediction[i]
        #check if probability greater than our randomly generated one
        if s >= r:
            #if it is, thats the likely next char
            char_id = i
            break
    #dont try to rank, just differentiate
    #initialize the vector
    char_one_hot = np.zeros(shape=[char_size])
    #that characters ID encoded
    #https://image.slidesharecdn.com/latin-150313140222-conversion-gate01/95/representation-learning-of-vectors-of-words-and-phrases-5-638.jpg?cb=1426255492
    char_one_hot[char_id] = 1.0
    return char_one_hot

In [16]:
#vectorize our data to feed it into model
len_per_section = 50
skip = 100
sections = []
next_chars = []

In [17]:
for i in range(0, len(text) - len_per_section, skip):
    sections.append(text[i: i + len_per_section])
    next_chars.append(text[i + len_per_section])

In [18]:
#matrix of section length by num of characters
X = np.zeros((len(sections), len_per_section, char_size))
#label column for all the character id's, still zero
y = np.zeros((len(sections), char_size))

In [19]:
#for each char in each section, convert each char to an ID
#for each section convert the labels to ids 
for i, section in enumerate(sections):
    for j, char in enumerate(section):
        X[i, j, char2id[char]] = 1
    y[i, char2id[next_chars[i]]] = 1
print(y)

[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  1.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  1.  0. ...,  0.  0.  0.]
 [ 0.  1.  0. ...,  0.  0.  0.]
 [ 0.  1.  0. ...,  0.  0.  0.]]


In [22]:
X.shape, y.shape

((12886, 50, 259), (12886, 259))

In [25]:
batch_size = 100
#total iterations
max_steps = 72001
#how often to log?
log_every = 100
#how often to save?
save_every = 6000
#too few and underfitting
#Underfitting occurs when there are too few neurons 
#in the hidden layers to adequately detect the signals in a complicated data set.
#too many and overfitting
hidden_nodes = 1024
#starting text
test_start = 'I am thinking that'
#to save our model
checkpoint_directory = 'ckpt'

In [26]:
if tf.gfile.Exists(checkpoint_directory):
    tf.gfile.DeleteRecursively(checkpoint_directory)
tf.gfile.MakeDirs(checkpoint_directory)

print('training data size:', len(X))
print('approximate steps per epoch:', int(len(X)/batch_size))


training data size: 12886
approximate steps per epoch: 128


In [27]:
# build the model
# computational graph
graph = tf.Graph()

with graph.as_default():
    # variables are placeholders
    # global_step = # of batches seen by the graph
    # everytime a batch is provided, the weights are updatd in the direction
    # that minimize the loss
    # global_step keeps track of # of batches seen so far
    
    global_step = tf.Variable(0)
    
